[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mravanba/comp551-notebooks/blob/master/CurseOfDimensionality.ipynb)


# Curse of Dimensionality

Some learning algorithms use a distance function to measure the similarity or dissimilarity of instances. Having many features corresponds to the instance "living" in a high-dimensional space. High dimensional data introduce difficulties, so called curse of dimensionality. However, by making some strong assumption about the data machine learning methods overcome these difficulties. First lets see the problems associated with high-dimensional data:

1. To "fill" a high dimensional space we need exponentially more samples.
- in one dimension, it is enough to have 9 labeled examples in the range (0,10), to make sure any new observation will be at a distance of `1` to our training data.
- in two dimensions, we need more than $9 \times 9$ labeled examples to satisfy the same condition.
- the number of required samples is $\mathcal{O}(9^D)$, where $D$ is the dimension.

2. In high dimension, it becomes harder to identify close neighbours. Let us demonstrate this with an example. We will generate random instances in various dimensions and measure the pairwise distance between these examples. 

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
# Set random seed for reproducibility
np.random.seed(1234)

In [ ]:
N = 1000  # Number of randomly generated points
Ds = [1, 2, 8, 32, 128, 784]  # Different dimensions to test

# Create subplots for each dimension
fig, axes = plt.subplots(ncols=len(Ds), nrows=1, constrained_layout=True, figsize=(len(Ds)*3, 3))

for i, D in enumerate(Ds):
    # Generate N random points in D-dimensional space
    # x has shape (N, D) - each row is a D-dimensional point
    # Points are uniformly distributed in [0, 1]^D
    x = np.random.rand(N, D)
    
    # Compute pairwise Euclidean distances using broadcasting
    # x[None,:,:] has shape (1, N, D) - adds a dimension at the front
    # x[:,None,:] has shape (N, 1, D) - adds a dimension in the middle
    # Broadcasting: (1, N, D) - (N, 1, D) = (N, N, D)
    # Result: dist[i,j] = distance between point i and point j
    # Subtraction creates all pairwise differences
    # Square, sum over features (axis -1), then square root
    dist = np.sqrt(np.sum((x[None, :, :] - x[:, None, :])**2, -1))
    
    # Plot histogram of all pairwise distances
    # dist.ravel() flattens (N, N) matrix to 1D array of all distances
    axes[i].hist(dist.ravel(), bins=100)
    axes[i].set_xlabel("pairwise distance")
    axes[i].set_ylabel("frequency")
    axes[i].set_title(f'{D} dimensions')

plt.show()

It is evident from this plot that, very unintuitively, in high dimensions, most points in the space have similar distances with each other!
If our datasets had a similar behaviour this could undermine learning in high-dimensions:
if all instances are more or less similar to each other on what basis can we label them differently?
Let's create similar plots for the `MNIST` dataset. For each dimension `D` we randomly select subset of dimensions from the `28 x 28 = 784` dimensions, and use them to measure the pairwise distance between examples.

In [ ]:
# Load the MNIST dataset
# 70,000 images of handwritten digits (0-9)
# Each image is 28x28 pixels = 784 features
from sklearn.datasets import fetch_openml
x_org, y = fetch_openml('mnist_784', version=1, return_X_y=True)

Lets see what the data looks like

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

def plot_digits(data):
    """
    Plot a grid of digit images from MNIST.
    
    Parameters:
    data: array of flattened images, shape (num_images, 784)
    """
    num_plots = data.shape[0]
    
    fig = plt.figure(figsize=(num_plots, 10.*num_plots))
    
    # Create a grid layout for displaying multiple images
    # nrows_ncols=(1, num_plots) means 1 row, num_plots columns
    grid = ImageGrid(fig, 111, nrows_ncols=(1, num_plots), axes_pad=0.1)
    
    for i in range(num_plots):
        # Reshape flattened image (784,) back to 2D (28, 28) for display
        grid[i].imshow(data[i].reshape((28, 28)))
    
    plt.show()

# Display the first 20 digits from the dataset
plot_digits(x_org[:20])

Now let's do the same measurement of pairwise distance, this time between instances in our dataset

In [ ]:
N = 1000  # Number of examples to use (for faster computation)

# Flatten and prepare the data
# x_org[:N] takes first N images
# reshape with -1 infers the dimension: (N, 28, 28) -> (N, 784)
x = np.reshape(x_org[:N], (N, -1))

# Normalize pixel values to [0, 1] range
# Original values are in [0, 255]
x = x / np.max(x)

# Test with different numbers of dimensions
Ds = [32, 128, 784]

fig, axes = plt.subplots(ncols=len(Ds), nrows=1, constrained_layout=True, figsize=(len(Ds)*3, 3))

# Randomly permute dimension indices to select random subsets of features
# This creates a random ordering of indices [0, 1, 2, ..., 783]
dim_inds = np.random.permutation(x.shape[1])

for i, D in enumerate(Ds):
    # Select D random dimensions from the data
    # dim_inds[:D] gives the first D indices from the permutation
    # x[:, dim_inds[:D]] selects those D columns, result is (N, D)
    x_D = x[:, dim_inds[:D]]
    
    # Compute pairwise Euclidean distances
    # Same broadcasting trick as before:
    # x_D[None,:,:] is (1, N, D)
    # x_D[:,None,:] is (N, 1, D)
    # Result dist is (N, N) where dist[i,j] = distance between sample i and j
    dist = np.sqrt(np.sum((x_D[None, :, :] - x_D[:, None, :])**2, -1))
    
    # Plot histogram of all pairwise distances
    axes[i].hist(dist.ravel(), bins=100)
    axes[i].set_xlabel("pairwise distance")
    axes[i].set_ylabel("frequency")
    axes[i].set_title(f'{D} dimensions')

plt.show()

This difference between real-world data and random data is explained by the **manifold hypothesis**. This hypothesis postulates that real-world data often reside close to high-dimensional *manifold*. Dimensionality reduction methods (aka manifold learning) try to estimate these low-dimensional encoding of the data. However, the point we wanted to show here is that because of this special behaviour, methods such as KNN continue to work with high-dimensional data.

## Another demonstration
We put one hypercube inside another with unit length, such that they have one common corner. We then place a large number of points regularly inside the larger hypercube.
Here again we change the dimension of the hypercubes and plot the portion of the points in the large hypercube that are also in the small hypercube (think of this as the portion of their volume) as we change the length of the side of the smaller hypercube.

In [ ]:
Ds = [1, 2, 8, 128, 512, 784]  # Different dimensions to test
N = 10000  # Number of random points to generate
grid_size = 1000  # Number of different cube sizes to test

# Generate grid of cube side lengths from 0 to 1
grid_points = np.linspace(0, 1, grid_size, endpoint=True)

# Store results: result[dimension_index, cube_size_index]
result = np.zeros((len(Ds), grid_size))

for j, D in enumerate(Ds):
    # Generate N random points in D-dimensional unit hypercube [0,1]^D
    # x has shape (N, D)
    x = np.random.rand(N, D)
    
    for i, g in enumerate(grid_points):
        # Count points where ALL coordinates are less than g
        # np.all(x < g, axis=1) checks if all D coordinates of each point are < g
        # This gives shape (N,) boolean array
        # Points satisfying this condition are inside the smaller cube [0,g]^D
        # Sum gives count, divide by N to get proportion
        result[j, i] = np.sum(np.all(x < g, axis=1)) / N
    
    # Plot the proportion of points inside as a function of cube side length
    plt.plot(grid_points, result[j, :], label=f'D={D}')

plt.xlabel('side of the cube')
plt.ylabel('portion of the points inside')
plt.legend()
plt.title('Hypercube Volume: Curse of Dimensionality')
plt.grid(True, alpha=0.3)
plt.show()

that is to get 1% of the points to be inside the inner cube we need to have a side of length .993! This is another expression of the idea that in high dimensions, *the mass is mostly at the corners!*

In [ ]:
# Find the cube side length needed to capture 1% of the points
# For each dimension, find where result is closest to 0.01
# np.argmin(np.abs(result - .01), axis=1) finds the index along axis 1 (cube sizes)
# Divide by grid_size to convert index back to actual side length
side_length = np.argmin(np.abs(result - .01), axis=1) / grid_size

# Plot how required side length changes with dimension
plt.plot(Ds, side_length, marker='o', linewidth=2, markersize=8)
plt.xlabel('dimension')
plt.ylabel('side of the sub-cube with 1% of the data')
plt.title('Required Cube Size for 1% of Points')
plt.grid(True, alpha=0.3)
plt.show()

print(f'To get 1% of points inside the inner cube for D=784 we need the side of the cube to be of length {side_length[-1]:.3f}')
print(f'This means the cube must have {side_length[-1]*100:.1f}% of the maximum side length!')